<span style="font-size:20px; font-weight:bold;">Tutorial 1: Aspen-Python-Interface </span>

To enable the understanding of the subsequent discussion of the Aspen-Python interface, some comments regarding the solvent-based carbon capture process using monoethanolamine (MEA) are first be provided:
* For a general description of the process, please refer to the publication associated with this repository.
* This process was specifically selected to illustrate the challenges involved in full automation of the developed framework.
* Aspen Plus file used to illustrate the framework: "Post_combustion_solvent_based_MEA.bkp".
* When interacting with Aspen Plus, the names of streams and process units has to be known. Thus, to facilitate reading the code, the process flowsheet implemented in the provided Aspen Plus file is shown below. 
* As the solvent cycle complicates reaching convergence in Aspen Plus, the loop is not closed. Instead, the stream of the lean solvent stream entering the absorber in the top is "cut" into two streams, LEANMEA and LEANMEAC. As a consequence, when conducting simulations, it has to be ensured that the flows of these two streams match each other.
* To ensure comparability, in this example the CO<sub>2</sub> removal efficiency is kept constant at 95 % by a design specification in Aspen Plus, related to streams FLUEGAS and CO2-OUT. The design specification is met by varying the required reboiler duty for the stripper.

![Alt-Text](Figures/Aspen_Plus_Flowsheet.png)

To showcase the most important functions of the Aspen-Python-Interface based on Zihao Wang (https://github.com/zwang1995/Aspen-Plus-Automation), a converged Aspen Plus file "Post_combustion_solvent_based_MEA.bkp" is opened and the initial performance of the process is returned. Then, the CO<sub>2</sub> concentration of the FLUEGAS stream is changed, the simulation is executed and the change in performance is reviewed.

First, we import the library aspen_utils (developed by Mr. Wang) to facilitate the interaction of Python with Aspen Plus. Ensure that all the packages such as os, time and win32com.client required for the library to work are installed in your Python distribution. Then, it is recommended to get familiar with the structure of the library. It mainly consists of one class called Aspen_Plus_Interface(), in which several functions to interact with Aspen Plus are defined. In the actual code below, we will first generate an instance of this class and then use some of its functions for our purpose. 

In [7]:
from aspen_utils import *

# Create instance of the Aspen_Plus_Interface() class
Aspen_Plus = Aspen_Plus_Interface() 

# Use load_bkp function to open your Aspen Plus file.
Aspen_Plus.load_bkp(r"Aspen_File/Post_combustion_solvent_based_MEA.bkp", 0, 1)
        #Load a process via bkp file
        #:param bkp_file: location of the Aspen Plus file
        #:param visible_state: Aspen Plus user interface, 0 is invisible and 1 is visible
        #:param dialog_state: Aspen Plus dialogs, 0 is not to suppress and 1 is to suppress
print("\n")

Aspen Plus 40.0 OLE Services




After running the code above you should get a notification with something like "Aspen Plus 40.0 OLE Services". It is recommended to work with the backup file of Aspen Plus (.bkp), since it has been observed that the interface is less prone to errors when establishing a connection. 

Furthermore, as the solvent-based process under consideration exhibit a high degree of complexity (due to the electrolyte chemistry involved), the first simulation in the Aspen Plus file "Post_combustion_solvent_based_MEA.bkp" was carried out manually in advance to establish a stable starting point. Thus, initial results should be available and we can take a look at it.  

To extract a value from Aspen Plus and assign it to a variable in Python, we first need to figure out, how Aspen Plus declares this variable in the simulation. You can do that within Aspen Plus by the <span style="font-weight:bold;">Variable Explorer: </span>

![Alt-Text](Figures/Aspen_Plus_Variable_Explorer.png)

For example, if we want to extract the value of the reboiler duty required for the stripper, we find the file path in the Variable Explorer like this: 

![Alt-Text](Figures/Aspen_Plus_Variable_Explorer2.png)

With this approach, most of the process parameters can be extracted from Aspen Plus with the command:

    Aspen_Plus.Application.Tree.FindNode(r"Filepath_Variable_Explorer").Value 

To assess the performance of the initial process configuration, the reboiler duty per ton CO<sub>2</sub> captured is used. Typically, this value is in a range of 3.2 - 4.0 MJ/ton CO<sub>2</sub> with Monoethanolamine (MEA) as a solvent. Let's test it:

In [11]:
###################################################################################################################
# 1. Return performance of initial process parameter configuration
###################################################################################################################

print("1. Return performance of initial process parameter configuration")
print("------------------------------------------------------------------------------------------------------------\n")

print("Process data: ")

# Get the amount of CO2 in feed stream into absorber in kg/h (see Aspen Plus File - stream FLUEGAS)
FLUEGAS_CO2_massflow = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Output\STR_MAIN\MASSFLOW\MIXED\CO2").Value *3600
print("CO\u2082 mass flow in FLUEGAS stream: ", round(FLUEGAS_CO2_massflow,1), " kg/h")

# Get the amount of CO2 captured by the process in kg/h (see Aspen Plus File - stream CO2OUT)
CO2OUT_CO2CaptureRate = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\CO2-OUT\Output\STR_MAIN\MASSFLOW\MIXED\CO2").Value *3600
print("Captured amount of CO\u2082 in CO2OUT: ", round(CO2OUT_CO2CaptureRate,1), " kg/h")

# Get reboiler duty required for the stripper in kW
STRIPPER_ReboilerDuty = Aspen_Plus.Application.Tree.FindNode(r"\Data\Blocks\REBOILER\Output\QCALC").Value * 1e-03
print("Required reboiler duty in Stripper: ", round(STRIPPER_ReboilerDuty,1), " kW \n")

print("Performance: ")

# CO2 removal efficiency
CO2_removal_eff = (CO2OUT_CO2CaptureRate / FLUEGAS_CO2_massflow) * 100
print("CO\u2082 removal efficiency: ", round(CO2_removal_eff,2), " %")

# Calculate performance of initial process configuration
STRIPPER_SpecReboilerDuty = round((STRIPPER_ReboilerDuty / (CO2OUT_CO2CaptureRate / 3600)), 5)
print("Performance measured by reboiler duty per ton CO\u2082 captured: ", round(STRIPPER_SpecReboilerDuty,1), " MJ/ ton CO\u2082\n")

1. Return performance of initial process parameter configuration
------------------------------------------------------------------------------------------------------------

Process data: 
CO₂ mass flow in FLUEGAS stream:  909.8  kg/h
Captured amount of CO₂ in CO2OUT:  864.0  kg/h
Required reboiler duty in Stripper:  912.1  kW 

Performance: 
CO₂ removal efficiency:  94.96  %
Performance measured by reboiler duty per ton CO₂ captured:  3800.5  MJ/ ton CO₂



With a specific reboiler duty of approximately 3800 MJ per ton of CO<sub>2</sub>, we are on the right track, although there is still room for improvement. Lets see how an increase of the CO<sub>2</sub> content in the flue gas entering the absorber impacts this performance metric. 

First, we extract the initial CO<sub>2</sub> and N<sub>2</sub> content from Aspen Plus with the same approach as above. Similarly, we can increase the CO<sub>2</sub> concentration by 0.1 vol%. If we increase the percentage of one component (CO<sub>2</sub>), we must reduce the percentage of another component (N<sub>2</sub>) accordingly. 

One way to ensure comparability between the initial result and the increased CO<sub>2</sub> concentration is to keep the CO<sub>2</sub> removal efficieny constant. In Aspen Plus, this can be achieved by a so-called *design specification*, in which the target value for the CO<sub>2</sub> flow in the stream leaving the stripper at the top (CO2-OUT; enriched CO<sub>2</sub> stream; subsequently send to storage/ utilization) is specified by 95 % of the CO<sub>2</sub> flow in the FLUEGAS stream in the code. For more details to the design specification, please take a look at the provided Aspen Plus file.

In [15]:
###################################################################################################################
# 2. Modify process parameters
###################################################################################################################

print("2. Modify process parameters")
print("------------------------------------------------------------------------------------------------------------\n")

# Get initial CO2 and N2 concentration in FLUEGAS stream
Fluegas_CO2conc = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\CO2").Value * 100
print("Initial CO\u2082 concentration in FLUEGAS stream: ", round(Fluegas_CO2conc,1), " vol%")

Fluegas_N2conc = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\N2").Value * 100
print("Initial N\u2082 concentration in FLUEGAS stream: ", round(Fluegas_N2conc,1), " vol%\n")

# Increase CO2 concentration in the FLUEGAS stream by 0.1 vol% and adjust N2 concentration accordingly 
Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\CO2").Value = (Fluegas_CO2conc + 0.1)/100 
Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\N2").Value = (Fluegas_N2conc - 0.1)/100

# Check updated CO2 and N2 concentration in FLUEGAS stream
Fluegas_CO2conc = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\CO2").Value * 100
print("New CO\u2082 concentration in FLUEGAS stream: ", round(Fluegas_CO2conc,1), " vol%")

Fluegas_N2conc = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\N2").Value * 100
print("New N\u2082 concentration in FLUEGAS stream: ", round(Fluegas_N2conc,1), " vol%\n")

# Since the CO2 concentration in the FLUEGAS stream is changed, the design specification in Aspen Plus for keeping the 
# CO2 removal efficiency constant also needs to be adjusted. To do that, we run the simulation to obtain the ned CO2 mass flow in the feed gas stream

print("Simulation is running to adjust design specification. Please wait...\n")

# Run the process simulation
Aspen_Plus.run_simulation()
Aspen_Plus.check_run_completion()

print("Design specification: ")

# 1. Get the new value of CO2, which needs to be captured (95 % CO2 removal efficiency)
DS_TargetValue = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Output\MASSFLOW\MIXED\CO2").Value * 0.95 * 3600
print("Initial captured amount of CO\u2082 in CO2OUT: ", round(CO2OUT_CO2CaptureRate,1), " kg/h")
print("New value of CO\u2082 mass flow, which needs to be captured to obtain a CO\u2082 removal efficiency of 95 %: ", round(DS_TargetValue,2), " kg/h")

# 2. Adjust the Design Specification in Aspen Plus accordingly
Aspen_Plus.Application.Tree.FindNode(r"\Data\Flowsheeting Options\Design-Spec\DS-1\Input\EXPR2").Value = DS_TargetValue


2. Modify process parameters
------------------------------------------------------------------------------------------------------------

Initial CO₂ concentration in FLUEGAS stream:  10.3  vol%
Initial N₂ concentration in FLUEGAS stream:  74.7  vol%

New CO₂ concentration in FLUEGAS stream:  10.4  vol%
New N₂ concentration in FLUEGAS stream:  74.6  vol%

Simulation is running for adjusting design specification. Please wait...

Design specification: 
Initial captured amount of CO₂ in CO2OUT:  864.0  kg/h
New value of CO2 mass flow, which needs to be captured to obtain a CO₂ removal efficiency of 95 %:  881.51  kg/h


While in section 1 only 864 kg/h had to be captured to achieve a CO<sub>2</sub> capture rate of approximately 95%, it is shown that 881.51 kg/h of CO<sub>2</sub> must now be captured (due to the increased CO<sub>2</sub> concentration in the FLUEGAS feed stream) and the design specification is adjusted accordingly.

After checking the adjustment of the CO<sub>2</sub> and N<sub>2</sub> concentrations, we are ready to run the simulation with the interface. To provide insights into the simulation process, a rudimentary control structure is shown in the following as an example: Aspen Plus has stored various codes for the progress of a simulation (success, warning, error). These codes can also be extracted using the interface and displayed on the console using if statements.

In [16]:
###################################################################################################################
# 3. Simulation run and trouble monitoring
###################################################################################################################

print("3. Simulation run and trouble monitoring")
print("------------------------------------------------------------------------------------------------------------\n")
print("Simulation running. It can take a minute. Please wait...")

# Run the process simulation
Aspen_Plus.run_simulation()
Aspen_Plus.check_run_completion()

# Trouble monitoring
# Aspen gives these numbers as an output regarding the success of a simulation run. (They are based on hexadecimal codes).
sim_success = 9345  # Hexadecimal: 0x00002481
sim_warning = 9348  # Hexadecimal: 0x00002484
sim_error = 9376    # Hexadecimal: 0x000024A0

run_status = Aspen_Plus.Application.Tree.FindNode(r"\Data").AttributeValue(12)

if run_status == sim_success:
    print("No errors encountered. Ready to extract simulation results.\n")

    print("---- Results for adjusted CO\u2082 concentration: ----")
    
    # Extracting results as described in 1.
    New_STRIPPER_ReboilerDuty = Aspen_Plus.Application.Tree.FindNode(r"\Data\Blocks\REBOILER\Output\QCALC").Value * 1e-03
    print("Required reboiler duty in Stripper: ", round(New_STRIPPER_ReboilerDuty,1), " kW")
    
    New_CO2OUT_CO2CaptureRate = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\CO2-OUT\Output\STR_MAIN\MASSFLOW\MIXED\CO2").Value *3600
    print("Captured amount of CO\u2082: ", round(New_CO2OUT_CO2CaptureRate,1), " kg/h")
    
    New_STRIPPER_SpecReboilerDuty = round((New_STRIPPER_ReboilerDuty / (New_CO2OUT_CO2CaptureRate / 3600)), 5)
    print("Performance measured by reboiler duty per ton CO\u2082 captured: ", round(New_STRIPPER_SpecReboilerDuty,1), " MJ/ ton CO\u2082\n")

elif run_status == sim_warning:
    print("Warnings encountered.")
else:
    print("Errors encountered.")

# Close the Aspen Plus file 
Aspen_Plus.close_bkp()

3. Simulation run and trouble monitoring
------------------------------------------------------------------------------------------------------------

Simulation running. It can take a minute. Please wait...
No errors encountered. Ready to extract simulation results.

---- Results for adjusted CO₂ concentration: ----
Required reboiler duty in Stripper:  918.8  kW
Captured amount of CO₂:  881.5  kg/h
Performance measured by reboiler duty per ton CO₂ captured:  3752.6  MJ/ ton CO₂



Examining the results indicates that the required specific reboiler duty is reduced from 3800 MJ/ ton CO<sub>2</sub> to approximately 3752,6 MJ/ ton CO<sub>2</sub> by a slight increase of the CO<sub>2</sub> content in the flue gas. This result emphasizes the importance of conducting comprehensive sensitivity analyses to find optimal operating conditions.

However, please note that when investigating larger changes to process parameters, the mass balance of the LEANMEA and LEANMEAC streams must be matched to obtain results within a defined tolerance. Since in the final code, the CO<sub>2</sub> content in the flue gas is varied over a wider range, this balancing is implemented with the same underlying methodology discussed here. In general, the open loop with mass balance considerations is necessary, since closing the loop results in a time-consuming try-and-error process to obtain correct results - even when manually conducting the simulations. This fact is due to the complexity of the process and reflects the current challenges of Process Systems Engineering. 

Finally, it should be noted that, based on the run_status if-statement above, further control structures in the code are conceivable, when convergence problems are observed. For example, in some cases, the convergence problem could be solved by first performing the simulation with the Wegstein solver and then switching to the Broyden solver if errors occur.

To store our data, we can easily write them to an Excel file using pandas, once we have assigned a variable to a process parameter in the code:

In [17]:
import pandas as pd

#performance_list = [New_STRIPPER_ReboilerDuty, New_CO2OUT_CO2CaptureRate, New_STRIPPER_SpecReboilerDuty]
Performance_metrics = ["Reboiler duty [kW]", "Captured CO\u2082 [kg/h]", "Spec. reboiler duty [MJ/ton CO\u2082]"]

df_performance = pd.DataFrame(
    [
        [STRIPPER_ReboilerDuty, CO2OUT_CO2CaptureRate, STRIPPER_SpecReboilerDuty],
        [New_STRIPPER_ReboilerDuty, New_CO2OUT_CO2CaptureRate, New_STRIPPER_SpecReboilerDuty]
    ],
    columns=Performance_metrics
)
print(df_performance)

with pd.ExcelWriter(r"Performance.xlsx", mode="w" , engine="openpyxl") as writer:
    df_performance.to_excel(writer, header=True, sheet_name="Performance", startrow=0, startcol=0)

   Reboiler duty [kW]  Captured CO₂ [kg/h]  Spec. reboiler duty [MJ/ton CO₂]
0          912.132795           864.014569                        3800.48923
1          918.831826           881.458913                        3752.63614


With the ExcelWriter function of pandas, an Excel file called Performance.xlsx is created and the data are stored. 